## CKKS Scheme

In [2]:
# this should throw an error as the global_scale isn't defined yet
try:
    print("global_scale:", context.global_scale)
except ValueError:
    print("The global_scale isn't defined yet")
    
# you can define it to 2 ** 20 for instance
context.global_scale = 2 ** 20
print("global_scale:", context.global_scale)

The global_scale isn't defined yet
global_scale: 1048576.0


In [3]:
import tenseal as ts

# Setup TenSEAL context
context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=8192,
            coeff_mod_bit_sizes=[60, 40, 40, 60]
          )
context

In [4]:
context.generate_galois_keys()
context.global_scale = 2**40
print("global_scale:", context.global_scale)

global_scale: 1099511627776.0


In [5]:
v1 = [0, 1, 2, 3, 4]
v2 = [4, 3, 2, 1, 0]

# encrypted vectors
enc_v1 = ts.ckks_vector(context, v1)
enc_v2 = ts.ckks_vector(context, v2)

In [6]:
result = enc_v1 + enc_v2
print(result.decrypt())

[3.999999995603078, 4.000000000713544, 4.000000001337283, 4.000000000178163, 4.000000000169062]


In [7]:
result = enc_v1.dot(enc_v2)
print(result.decrypt())

[10.000002432731359]


In [8]:
matrix = [
  [  73,  0.5,   8],
  [  81,   -5,  66],
  [-100,  -78,  -2],
  [   0,    9,  17],
  [  69,   11,  10],
]

result = enc_v1.matmul(matrix)
print(result.decrypt())

[157.00002111403685, -90.00001203066218, 153.00002055522464]


In [9]:
import torch
from torchvision import transforms
from random import randint
import pickle
from PIL import Image
import numpy as np
from matplotlib.pyplot import imshow
from typing import Dict

import tenseal as ts

In [74]:
def context():
    context = ts.context(ts.SCHEME_TYPE.CKKS, 8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
    context.global_scale = pow(2, 40)
    context.generate_galois_keys()
    return context

context = context()

In [11]:
plain1 = ts.plain_tensor([1,2,3,4], [2,2])

print(" First tensor: Shape = {} Data = {}".format(plain1.shape, plain1.tolist()))

plain2 = ts.plain_tensor(np.array([5,6,7,8]).reshape(2,2))
print(" Second tensor: Shape = {} Data = {}".format(plain2.shape, plain2.tolist()))

 First tensor: Shape = [2, 2] Data = [[1.0, 2.0], [3.0, 4.0]]
 Second tensor: Shape = [2, 2] Data = [[5.0, 6.0], [7.0, 8.0]]


In [12]:
encrypted_tensor1 = ts.ckks_tensor(context, plain1)
encrypted_tensor2 = ts.ckks_tensor(context, plain2)

print(" Shape = {}".format(encrypted_tensor1.shape))
print(" Encrypted Data = {}.".format(encrypted_tensor1))


encrypted_tensor_from_np = ts.ckks_tensor(context, np.array([5,6,7,8]).reshape([2,2]))
print(" Shape = {}".format(encrypted_tensor_from_np.shape))

 Shape = [2, 2]
 Encrypted Data = <tenseal.tensors.ckkstensor.CKKSTensor object at 0x7fc77c8a6c10>.
 Shape = [2, 2]


In [13]:
def decrypt(enc):
    return enc.decrypt().tolist()

In [14]:
result = encrypted_tensor1 + encrypted_tensor2
print("Plain equivalent: {} + {}\nDecrypted result: {}.".format(plain1.tolist(), plain2.tolist(), decrypt(result)))

Plain equivalent: [[1.0, 2.0], [3.0, 4.0]] + [[5.0, 6.0], [7.0, 8.0]]
Decrypted result: [[6.000000000039379, 7.999999999362373], [9.99999999955967, 12.000000000077229]].


In [15]:
result = encrypted_tensor1 - encrypted_tensor2
print("Plain equivalent: {} - {}\nDecrypted result: {}.".format(plain1.tolist(), plain2.tolist(), decrypt(result)))

Plain equivalent: [[1.0, 2.0], [3.0, 4.0]] - [[5.0, 6.0], [7.0, 8.0]]
Decrypted result: [[-3.9999999995520876, -4.000000000433112], [-3.9999999997084887, -3.99999999979783]].


In [16]:
result = encrypted_tensor1 * encrypted_tensor2
print("Plain equivalent: {} * {}\nDecrypted result: {}.".format(plain1.tolist(), plain2.tolist(), decrypt(result)))

Plain equivalent: [[1.0, 2.0], [3.0, 4.0]] * [[5.0, 6.0], [7.0, 8.0]]
Decrypted result: [[5.000000671480139, 12.000001605212546], [21.000002814536774, 32.00000429271307]].


In [17]:
plain = ts.plain_tensor([5,6,7,8], [2,2])
result = encrypted_tensor1 * plain

print("Plain equivalent: {} * {}\nDecrypted result: {}.".format(plain1.tolist(), plain.tolist(), decrypt(result)))

Plain equivalent: [[1.0, 2.0], [3.0, 4.0]] * [[5.0, 6.0], [7.0, 8.0]]
Decrypted result: [[5.000000672140737, 12.000001605640215], [21.00000281658246, 32.00000429214441]].


In [18]:
result = -encrypted_tensor1 

print("Plain equivalent: -{}\nDecrypted result: {}.".format(plain1.tolist(), decrypt(result)))

Plain equivalent: -[[1.0, 2.0], [3.0, 4.0]]
Decrypted result: [[-1.0000000002436458, -1.9999999994646303], [-2.99999999992559, -4.000000000139699]].


In [19]:
result = encrypted_tensor1 ** 3
print("Plain equivalent: {} ^ 3\nDecrypted result: {}.".format(plain1.tolist(), decrypt(result)))

Plain equivalent: [[1.0, 2.0], [3.0, 4.0]] ^ 3
Decrypted result: [[1.0000008049443354, 8.000006430286314], [27.00002172329472, 64.00005150462951]].


In [20]:
result = encrypted_tensor1.polyval([1,0,1,1])

print("X = {}".format(plain1.tolist()))
print("1 + X^2 + X^3 = {}.".format(decrypt(result)))

X = [[1.0, 2.0], [3.0, 4.0]]
1 + X^2 + X^3 = [[3.000000938308724, 13.000006964338594], [37.00002292978027, 81.00005365123276]].


In [21]:
result = encrypted_tensor1.polyval([0.5, 0.197, 0, -0.004])


print("X = {}".format(plain1.tolist()))
print("0.5 + 0.197 X - 0.004 x^X = {}.".format(decrypt(result)))

X = [[1.0, 2.0], [3.0, 4.0]]
0.5 + 0.197 X - 0.004 x^X = [[0.6930000231493176, 0.8620000218942098], [0.9829999743629149, 1.0319998709409617]].


## Test CKKS parameters 

In [47]:
import tenseal as ts
import tenseal.sealapi as seal
import random
import pickle
import numpy as np
import struct
import sys
import math
import array
from IPython.display import HTML, display
import tabulate
import pytest

In [48]:
def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

# Type of Encryption
enc_type_str = {
    ts.ENCRYPTION_TYPE.SYMMETRIC : "symmetric", 
    ts.ENCRYPTION_TYPE.ASYMMETRIC : "asymmetric",
}

# Select Ecryption Scheme ( CKKS or BFV )
scheme_str = {
    ts.SCHEME_TYPE.CKKS : "ckks", 
    ts.SCHEME_TYPE.BFV : "bfv",
}

def decrypt(enc):
    return enc.decrypt()

## size of the context, depending on the input parameters.

In [49]:
ctx_size_benchmarks = [["Encryption Type", 
                        "Scheme Type", 
                        "Polynomial modulus", 
                        "Coefficient modulus sizes", 
                        "Saved keys", 
                        "Context serialized size", ]]

for enc_type in [ts.ENCRYPTION_TYPE.SYMMETRIC, ts.ENCRYPTION_TYPE.ASYMMETRIC]:
    for (poly_mod, coeff_mod_bit_sizes) in [
        (8192, [40, 20, 40]),
    ]:
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        context.generate_galois_keys()
        context.generate_relin_keys()
      
        ser = context.serialize(save_public_key=True, 
                                save_secret_key=True, 
                                save_galois_keys=True, 
                                save_relin_keys=True)
        ctx_size_benchmarks.append([enc_type_str[enc_type], 
                                    scheme_str[ts.SCHEME_TYPE.CKKS], 
                                    poly_mod, 
                                    coeff_mod_bit_sizes, 
                                    "all", 
                                    convert_size(len(ser))])
        
        if enc_type is ts.ENCRYPTION_TYPE.ASYMMETRIC:
            ser = context.serialize(save_public_key=True, 
                                    save_secret_key=False, 
                                    save_galois_keys=False, 
                                    save_relin_keys=False)
            ctx_size_benchmarks.append([enc_type_str[enc_type], 
                                        scheme_str[ts.SCHEME_TYPE.CKKS], 
                                        poly_mod, 
                                        coeff_mod_bit_sizes, 
                                        "Public key", 
                                        convert_size(len(ser))])
  
        ser = context.serialize(save_public_key=False, 
                                save_secret_key=True, 
                                save_galois_keys=False, 
                                save_relin_keys=False)
        ctx_size_benchmarks.append([enc_type_str[enc_type], 
                                    scheme_str[ts.SCHEME_TYPE.CKKS], 
                                    poly_mod, 
                                    coeff_mod_bit_sizes, 
                                    "Secret key",  
                                    convert_size(len(ser))])
                                    
        ser = context.serialize(save_public_key=False, 
                                save_secret_key=False, 
                                save_galois_keys=True, 
                                save_relin_keys=False)
        ctx_size_benchmarks.append([enc_type_str[enc_type], 
                                    scheme_str[ts.SCHEME_TYPE.CKKS], 
                                    poly_mod, 
                                    coeff_mod_bit_sizes, 
                                    "Galois keys",  
                                    convert_size(len(ser))])
                                                                      
        ser = context.serialize(save_public_key=False, 
                                save_secret_key=False, 
                                save_galois_keys=False, 
                                save_relin_keys=True)
        ctx_size_benchmarks.append([enc_type_str[enc_type], 
                                    scheme_str[ts.SCHEME_TYPE.CKKS], 
                                    poly_mod, 
                                    coeff_mod_bit_sizes, 
                                    "Relin keys",  
                                    convert_size(len(ser))])
  
        ser = context.serialize(save_public_key=False, 
                                save_secret_key=False, 
                                save_galois_keys=False, 
                                save_relin_keys=False)
        ctx_size_benchmarks.append([enc_type_str[enc_type], 
                                    scheme_str[ts.SCHEME_TYPE.CKKS], 
                                    poly_mod, 
                                    coeff_mod_bit_sizes, 
                                    "none", 
                                    convert_size(len(ser))])

display(HTML(tabulate.tabulate(ctx_size_benchmarks, tablefmt='html')))

Encryption Type,Scheme Type,Polynomial modulus,Coefficient modulus sizes,Saved keys,Context serialized size
symmetric,ckks,8192,"[40, 20, 40]",all,124.88 KB
symmetric,ckks,8192,"[40, 20, 40]",Secret key,124.88 KB
symmetric,ckks,8192,"[40, 20, 40]",Galois keys,11.89 MB
symmetric,ckks,8192,"[40, 20, 40]",Relin keys,503.56 KB
symmetric,ckks,8192,"[40, 20, 40]",none,91.0 B
asymmetric,ckks,8192,"[40, 20, 40]",all,376.15 KB
asymmetric,ckks,8192,"[40, 20, 40]",Public key,251.43 KB
asymmetric,ckks,8192,"[40, 20, 40]",Secret key,124.81 KB
asymmetric,ckks,8192,"[40, 20, 40]",Galois keys,11.89 MB
asymmetric,ckks,8192,"[40, 20, 40]",Relin keys,503.64 KB


In [31]:
data = [random.uniform(-10, 10) for _ in range(10 ** 3)]
network_data = pickle.dumps(data)
print("Plain data size in bytes {}".format(convert_size(len(network_data))))

Plain data size in bytes 8.8 KB


In [32]:
def display_param(test_param):
    enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC
    ct_size_benchmarks_ckks = [["Encryption Type", 
                       "Scheme Type", 
                       "Polynomial modulus", 
                       "Coefficient modulus sizes", 
                       "Precision", 
                       "Ciphertext serialized size", 
                       "Encryption increase ratio"
                      ]]

    for (poly_mod, coeff_mod_bit_sizes, prec) in test_parameters_ckks:
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        scale = 2 ** prec
        ckks_vec = ts.ckks_vector(context, data, scale)

        enc_network_data = ckks_vec.serialize()
        ct_size_benchmarks_ckks.append([enc_type_str[enc_type], 
                                   scheme_str[ts.SCHEME_TYPE.CKKS], 
                                   poly_mod, coeff_mod_bit_sizes, 
                                   "2**{}".format(prec),
                                   convert_size(len(enc_network_data)), 
                                   round(len(enc_network_data) / len(network_data), 2)
                                  ])
    display(HTML(tabulate.tabulate(ct_size_benchmarks_ckks, tablefmt='html')))    

In [33]:
test_parameters_ckks = [
    (2**13, [60, 40, 60], 40),
    (2**13, [60, 40, 60], 20),
    (2**13, [60, 40, 60], 10),
    (2**13, [60, 40, 60], 5),
    (2**13, [40, 20, 40], 40),
    (2**13, [40, 20, 40], 20),
    (2**13, [40, 20, 40], 10),
    (2**13, [40, 20, 40], 5),
    (2**13, [20, 20, 20], 20),
    (2**13, [20, 20, 20], 10),
    (2**13, [20, 20, 20], 5)
]
display_param(test_parameters_ckks)

Encryption Type,Scheme Type,Polynomial modulus,Coefficient modulus sizes,Precision,Ciphertext serialized size,Encryption increase ratio
asymmetric,ckks,8192,"[60, 40, 60]",2**40,229.77 KB,26.12
asymmetric,ckks,8192,"[60, 40, 60]",2**20,229.81 KB,26.12
asymmetric,ckks,8192,"[60, 40, 60]",2**10,229.82 KB,26.13
asymmetric,ckks,8192,"[60, 40, 60]",2**5,229.83 KB,26.13
asymmetric,ckks,8192,"[40, 20, 40]",2**40,153.35 KB,17.43
asymmetric,ckks,8192,"[40, 20, 40]",2**20,153.18 KB,17.41
asymmetric,ckks,8192,"[40, 20, 40]",2**10,154.49 KB,17.56
asymmetric,ckks,8192,"[40, 20, 40]",2**5,153.22 KB,17.42
asymmetric,ckks,8192,"[20, 20, 20]",2**20,104.78 KB,11.91
asymmetric,ckks,8192,"[20, 20, 20]",2**10,104.58 KB,11.89


In [34]:
test_parameters_ckks = [
    (2**13, [60, 60], 40),
    (2**13, [60, 60], 20),
    (2**13, [60, 60], 10),
    (2**13, [60, 60], 5),
    (2**13, [40, 40], 38),
    (2**13, [40, 40], 20),
    (2**13, [40, 40], 10),
    (2**13, [40, 40], 5),
    (2**13, [20, 20], 18),
    (2**13, [20, 20], 10),
    (2**13, [20, 20], 5),
]
display_param(test_parameters_ckks)

Encryption Type,Scheme Type,Polynomial modulus,Coefficient modulus sizes,Precision,Ciphertext serialized size,Encryption increase ratio
asymmetric,ckks,8192,"[60, 60]",2**40,128.13 KB,14.57
asymmetric,ckks,8192,"[60, 60]",2**20,128.13 KB,14.57
asymmetric,ckks,8192,"[60, 60]",2**10,128.13 KB,14.57
asymmetric,ckks,8192,"[60, 60]",2**5,128.13 KB,14.57
asymmetric,ckks,8192,"[40, 40]",2**38,92.07 KB,10.47
asymmetric,ckks,8192,"[40, 40]",2**20,92.12 KB,10.47
asymmetric,ckks,8192,"[40, 40]",2**10,92.21 KB,10.48
asymmetric,ckks,8192,"[40, 40]",2**5,92.14 KB,10.47
asymmetric,ckks,8192,"[20, 20]",2**18,52.82 KB,6.0
asymmetric,ckks,8192,"[20, 20]",2**10,52.7 KB,5.99


In [35]:
test_parameters_ckks = [
    (2**12, [40, 20, 40], 40),
    (2**12, [40, 20, 40], 20),
    (2**12, [40, 20, 40], 10),
    (2**12, [40, 20, 40], 5),
    (2**12, [30, 20, 30], 40),
    (2**12, [30, 20, 30], 20),
    (2**12, [30, 20, 30], 10),
    (2**12, [30, 20, 30], 5),
    (2**12, [20, 20, 20], 20),
    (2**12, [20, 20, 20], 10),
    (2**12, [20, 20, 20], 5)
]
display_param(test_parameters_ckks)

Encryption Type,Scheme Type,Polynomial modulus,Coefficient modulus sizes,Precision,Ciphertext serialized size,Encryption increase ratio
asymmetric,ckks,4096,"[40, 20, 40]",2**40,79.07 KB,8.99
asymmetric,ckks,4096,"[40, 20, 40]",2**20,79.03 KB,8.98
asymmetric,ckks,4096,"[40, 20, 40]",2**10,78.97 KB,8.98
asymmetric,ckks,4096,"[40, 20, 40]",2**5,78.96 KB,8.98
asymmetric,ckks,4096,"[30, 20, 30]",2**40,66.4 KB,7.55
asymmetric,ckks,4096,"[30, 20, 30]",2**20,66.33 KB,7.54
asymmetric,ckks,4096,"[30, 20, 30]",2**10,66.38 KB,7.55
asymmetric,ckks,4096,"[30, 20, 30]",2**5,66.36 KB,7.54
asymmetric,ckks,4096,"[20, 20, 20]",2**20,53.33 KB,6.06
asymmetric,ckks,4096,"[20, 20, 20]",2**10,53.25 KB,6.05


In [36]:
test_parameters_ckks = [
    (2**12, [30, 30], 28),
    (2**12, [30, 30], 20),
    (2**12, [30, 30], 10),
    (2**12, [30, 30], 5),
    (2**11, [20, 20], 18),
    (2**11, [20, 20], 10),
    (2**11, [20, 20], 5),
    (2**11, [16, 16], 10),
    (2**11, [16, 16], 5)
]
display_param(test_parameters_ckks)

Encryption Type,Scheme Type,Polynomial modulus,Coefficient modulus sizes,Precision,Ciphertext serialized size,Encryption increase ratio
asymmetric,ckks,4096,"[30, 30]",2**28,34.27 KB,3.9
asymmetric,ckks,4096,"[30, 30]",2**20,34.26 KB,3.9
asymmetric,ckks,4096,"[30, 30]",2**10,34.24 KB,3.89
asymmetric,ckks,4096,"[30, 30]",2**5,34.23 KB,3.89
asymmetric,ckks,2048,"[20, 20]",2**18,13.23 KB,1.5
asymmetric,ckks,2048,"[20, 20]",2**10,13.08 KB,1.49
asymmetric,ckks,2048,"[20, 20]",2**5,13.04 KB,1.48
asymmetric,ckks,2048,"[16, 16]",2**10,9.43 KB,1.07
asymmetric,ckks,2048,"[16, 16]",2**5,9.42 KB,1.07


In [63]:
data = [ random.random()]

enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC
ct_size_benchmarks = [["Value range", 
                       "Polynomial modulus", 
                       "Coefficient modulus sizes", 
                       "Precision", 
                       "Operation", 
                       "Status"]]


for data_pow in [-1, 0, 1, 5, 11, 21, 41, 51]:
    data = [ random.uniform(2 ** data_pow, 2 ** (data_pow + 1))]
    for (poly_mod, coeff_mod_bit_sizes, prec) in [
        (8192, [60, 40, 60], 40),
        (8192, [40, 20, 40], 40)
    ]:
        val_str = "[2^{} - 2^{}]".format(data_pow, data_pow + 1)
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        scale = 2 ** prec
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
        except BaseException as e:
            ct_size_benchmarks.append([val_str, 
                                       poly_mod, 
                                       coeff_mod_bit_sizes, 
                                       "2**{}".format(prec), 
                                       "encrypt", 
                                       "encryption failed"])
            continue
    
        decrypted = decrypt(ckks_vec)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == data:
                ct_size_benchmarks.append([val_str, 
                                           poly_mod, 
                                           coeff_mod_bit_sizes,
                                           "2**{}".format(prec), 
                                           "encrypt", 
                                           "decryption prec 2 ** {}".format(-dec_prec)])
                break
        ckks_sum = ckks_vec + ckks_vec
        decrypted = decrypt(ckks_sum)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] + data[0]]:
                ct_size_benchmarks.append([val_str, 
                                           poly_mod, 
                                           coeff_mod_bit_sizes, 
                                           "2**{}".format(prec), 
                                           "sum", 
                                           "decryption prec 2 ** {}".format(-dec_prec)])
                break
                
# We add more depth for the multiplication scenario
for data_pow in [-1, 0, 1, 5, 11, 21, 41, 51]:
    data = [ random.uniform(2 ** data_pow, 2 ** (data_pow + 1))]
    for (poly_mod, coeff_mod_bit_sizes, prec) in [
        (8192, [60, 40, 60], 40),
        (8192, [40, 20, 40], 40)
    ]:
        val_str = "[2^{} - 2^{}]".format(data_pow, data_pow + 1)
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        scale = 2 ** prec
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
        except BaseException as e:
            continue
                
        try:
            ckks_mul = ckks_vec * ckks_vec
        except:
            ct_size_benchmarks.append([val_str, 
                                       poly_mod, 
                                       coeff_mod_bit_sizes, 
                                       "2**{}".format(prec), 
                                       "mul", 
                                       "failed"])
            continue
        decrypted = decrypt(ckks_mul)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] * data[0]]:
                ct_size_benchmarks.append([val_str, 
                                           poly_mod, 
                                           coeff_mod_bit_sizes, 
                                           "2**{}".format(prec), 
                                           "mul", 
                                           "decryption prec 2 ** {}".format(-dec_prec)])
                break

display(HTML(tabulate.tabulate(ct_size_benchmarks, tablefmt='html')))

Value range,Polynomial modulus,Coefficient modulus sizes,Precision,Operation,Status
[2^-1 - 2^0],8192,"[60, 40, 60]",2**40,encrypt,decryption prec 2 ** -34
[2^-1 - 2^0],8192,"[60, 40, 60]",2**40,sum,decryption prec 2 ** -33
[2^-1 - 2^0],8192,"[40, 20, 40]",2**40,encrypt,decryption prec 2 ** -31
[2^-1 - 2^0],8192,"[40, 20, 40]",2**40,sum,decryption prec 2 ** -30
[2^0 - 2^1],8192,"[60, 40, 60]",2**40,encrypt,decryption prec 2 ** -31
[2^0 - 2^1],8192,"[60, 40, 60]",2**40,sum,decryption prec 2 ** -30
[2^0 - 2^1],8192,"[40, 20, 40]",2**40,encrypt,decryption prec 2 ** -32
[2^0 - 2^1],8192,"[40, 20, 40]",2**40,sum,decryption prec 2 ** -31
[2^1 - 2^2],8192,"[60, 40, 60]",2**40,encrypt,decryption prec 2 ** -33
[2^1 - 2^2],8192,"[60, 40, 60]",2**40,sum,decryption prec 2 ** -32


In [78]:
def decrypt(enc):
    return enc.decrypt().tolist()

In [95]:
plain3 = ts.plain_tensor([1,2,3,4], [2,2])
encrypted_tensor3 = ts.ckks_tensor(context, plain3)
results = encrypted_tensor3.polyval([-145,48,-34,3])
print("3x3-34x2+18x-145 = {}.".format(decrypt(result)))

3x3-34x2+18x-145 = [[3.000000939856626, 13.00000696384038], [37.00002292958378, 81.00005365138446]].


In [110]:
import torch
from torchvision import datasets
import torchvision.transforms as transforms
import numpy as np

torch.manual_seed(73)

train_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST('data', train=False, download=True, transform=transforms.ToTensor())

batch_size = 64

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [111]:
class ConvNet(torch.nn.Module):
    def __init__(self, hidden=64, output=10):
        super(ConvNet, self).__init__()        
        self.conv1 = torch.nn.Sequential(torch.nn.Conv2d(1, 4, kernel_size=7, padding=0, stride=3),
                                         torch.nn.ReLU())
        self.fc1 = torch.nn.Sequential(torch.nn.Linear(256, hidden),
                                       torch.nn.ReLU())
        self.fc2 = torch.nn.Linear(hidden, output)

    def forward(self, x):
        x = self.conv1(x)
        # the model uses the square activation function
        # x = x * x
        # flattening while keeping the batch axis
        x = x.view(-1, 256)
        x = self.fc1(x)
        # x = x * x
        x = self.fc2(x)
        return x

In [112]:
def train(model, train_loader, criterion, optimizer, n_epochs=10):
    # model in training mode
    model.train()
    for epoch in range(1, n_epochs+1):

        train_loss = 0.0
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # calculate average losses
        train_loss = train_loss / len(train_loader)

        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))
    
    # model in evaluation mode
    model.eval()
    return model

In [113]:
model = ConvNet()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model = train(model, train_loader, criterion, optimizer, 10)

Epoch: 1 	Training Loss: 0.397367
Epoch: 2 	Training Loss: 0.164987
Epoch: 3 	Training Loss: 0.124856
Epoch: 4 	Training Loss: 0.102274
Epoch: 5 	Training Loss: 0.086198
Epoch: 6 	Training Loss: 0.075736
Epoch: 7 	Training Loss: 0.068237
Epoch: 8 	Training Loss: 0.062010
Epoch: 9 	Training Loss: 0.057101
Epoch: 10 	Training Loss: 0.051869


In [114]:
def test(model, test_loader, criterion):
    # initialize lists to monitor test loss and accuracy
    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    # model in evaluation mode
    model.eval()

    for data, target in test_loader:
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss.item()
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)
        # compare predictions to true label
        correct = np.squeeze(pred.eq(target.data.view_as(pred)))
        # calculate test accuracy for each object class
        for i in range(len(target)):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

    # calculate and print avg test loss
    test_loss = test_loss/len(test_loader)
    print(f'Test Loss: {test_loss:.6f}\n')

    for label in range(10):
        print(
            f'Test Accuracy of {label}: {int(100 * class_correct[label] / class_total[label])}% '
            f'({int(np.sum(class_correct[label]))}/{int(np.sum(class_total[label]))})'
        )

    print(
        f'\nTest Accuracy (Overall): {int(100 * np.sum(class_correct) / np.sum(class_total))}% ' 
        f'({int(np.sum(class_correct))}/{int(np.sum(class_total))})'
    )
    
test(model, test_loader, criterion)

Test Loss: 0.075659

Test Accuracy of 0: 98% (967/980)
Test Accuracy of 1: 99% (1125/1135)
Test Accuracy of 2: 98% (1014/1032)
Test Accuracy of 3: 98% (994/1010)
Test Accuracy of 4: 98% (968/982)
Test Accuracy of 5: 97% (869/892)
Test Accuracy of 6: 97% (937/958)
Test Accuracy of 7: 98% (1012/1028)
Test Accuracy of 8: 96% (941/974)
Test Accuracy of 9: 93% (945/1009)

Test Accuracy (Overall): 97% (9772/10000)
